##### Context and Content

A company which is active in Big Data and Data Science wants to hire data scientists among people who successfully pass some courses which conduct by the company. Many people signup for their training. Company wants to know which of these candidates are really wants to work for the company after training or looking for a new employment because it helps to reduce the cost and time as well as the quality of training or planning the courses and categorization of candidates. Information related to demographics, education, experience are in hands from candidates signup and enrollment.

This dataset designed to understand the factors that lead a person to leave current job for HR researches too. By model(s) that uses the current credentials,demographics,experience data you will predict the probability of a candidate to look for a new job or will work for the company, as well as interpreting affected factors on employee decision.

The whole data divided to train and test . Target isn't included in test but the test target values data file is in hands for related tasks. A sample submission correspond to enrollee_id of test set provided too with columns : enrollee _id , target


##### Features

enrollee_id : Unique ID for candidate

city: City code

city_ development _index : Developement index of the city (scaled)

gender: Gender of candidate

relevent_experience: Relevant experience of candidate

enrolled_university: Type of University course enrolled if any

education_level: Education level of candidate

major_discipline :Education major discipline of candidate

experience: Candidate total experience in years

company_size: No of employees in current employer's company

company_type : Type of current employer

lastnewjob: Difference in years between previous job and current job

training_hours: training hours completed

target: 0 – Not looking for job change, 1 – Looking for a job change

Inspiration
Predict the probability of a candidate will work for the company
Interpret model(s) such a way that illustrate which features affect candidate decision

##### Goal of the project

Prepare the model to predict whether someone is willing to change the job.

##### Minor goals
- Investigate the data
- Understand the data - search for insights, clues
- Preprocess the data, so it can be used later on by a machine learning model

In [2]:
# import the pandas library

import pandas as pd

In [3]:
# read the file with training data

file = 'data/aug_train.csv'
df = pd.read_csv(file)

In [4]:
# let's see the first 5 rows of the dataset

df.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


##### Examining the data

We need to get a grasp of how big the dataset is. Also, we need to know how many cases we have for both cases we want to predict:
- people who want to change their job
- people who want to stay at their current workplace

In [5]:
df.describe()

,enrollee_id,city_development_index,training_hours,target
count,19158.000000,19158.000000,19158.000000,19158.000000
mean,16875.358179,0.828848,65.366896,0.249348
std,9616.292592,0.123362,60.058462,0.432647
min,1.000000,0.448000,1.000000,0.000000
25%,8554.250000,0.740000,23.000000,0.000000
50%,16982.500000,0.903000,47.000000,0.000000
75%,25169.750000,0.920000,88.000000,0.000000
max,33380.000000,0.949000,336.000000,1.000000


# Question: Did this output tell us anything? Conclusions?

In [6]:
df.shape

(19158, 14)

# Question: Is this enough? Can we expect to have our algorithm work properly? What else do we want to check?

In [7]:
df['target'] == 0

0        False
1         True
2         True
3        False
4         True
         ...  
19153    False
19154    False
19155     True
19156     True
19157     True
Name: target, Length: 19158, dtype: bool

In [8]:
df['target'] == 1

0         True
1        False
2        False
3         True
4        False
         ...  
19153     True
19154     True
19155    False
19156    False
19157    False
Name: target, Length: 19158, dtype: bool

In [9]:
df[df['target'] == 0].shape

(14381, 14)

In [10]:
df[df['target'] == 1].shape

(4777, 14)

# Question: What do we notice here? What does it tell us? Why it's important from the algorithm perspective?

### Investigation of columns
Let's investigate separate columns one by one.

##### Two ways of manipulating pandas dataframes:
- assign output of a function to a variable
- set *inplace* flag to True

In [11]:
# 1) sorted_df = df.set_index('enrollee_id').sort_index()
# 2) df.set_index('enrollee_id', inplace=True)

In [12]:
df = df.set_index('enrollee_id')

In [13]:
df.head()

,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
enrollee_id,,,,,,,,,,,,,
8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [14]:
df = df.sort_index()

In [15]:
df.head()

,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
enrollee_id,,,,,,,,,,,,,
1,city_103,0.920,Male,No relevent experience,no_enrollment,High School,NaN,2,NaN,Pvt Ltd,never,150,0.0
2,city_103,0.920,Male,No relevent experience,no_enrollment,Phd,STEM,15,10000+,NGO,3,128,1.0
4,city_103,0.920,Male,No relevent experience,no_enrollment,Masters,STEM,9,50-99,Public Sector,>4,13,0.0
5,city_67,0.855,Male,Has relevent experience,no_enrollment,Masters,STEM,10,<10,Early Stage Startup,1,12,0.0
7,city_136,0.897,Male,Has relevent experience,Full time course,Masters,STEM,13,NaN,NaN,1,18,0.0


# Question: Look at the rest of the columns: please answer what dtypes do their represent?

### Let's investigate the unique values for columns containing categorical variables

In [15]:
df.columns

Index(['city', 'city_development_index', 'gender', 'relevent_experience',
       'enrolled_university', 'education_level', 'major_discipline',
       'experience', 'company_size', 'company_type', 'last_new_job',
       'training_hours', 'target'],
      dtype='object')

In [16]:
df.head()

,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
enrollee_id,,,,,,,,,,,,,
1,city_103,0.920,Male,No relevent experience,no_enrollment,High School,NaN,2,NaN,Pvt Ltd,never,150,0.0
2,city_103,0.920,Male,No relevent experience,no_enrollment,Phd,STEM,15,10000+,NGO,3,128,1.0
4,city_103,0.920,Male,No relevent experience,no_enrollment,Masters,STEM,9,50-99,Public Sector,>4,13,0.0
5,city_67,0.855,Male,Has relevent experience,no_enrollment,Masters,STEM,10,<10,Early Stage Startup,1,12,0.0
7,city_136,0.897,Male,Has relevent experience,Full time course,Masters,STEM,13,NaN,NaN,1,18,0.0


In [17]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19158 entries, 1 to 33380
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city                    19158 non-null  object 
 1   city_development_index  19158 non-null  float64
 2   gender                  14650 non-null  object 
 3   relevent_experience     19158 non-null  object 
 4   enrolled_university     18772 non-null  object 
 5   education_level         18698 non-null  object 
 6   major_discipline        16345 non-null  object 
 7   experience              19093 non-null  object 
 8   company_size            13220 non-null  object 
 9   company_type            13018 non-null  object 
 10  last_new_job            18735 non-null  object 
 11  training_hours          19158 non-null  int64  
 12  target                  19158 non-null  float64
dtypes: float64(2), int64(1), object(10)
memory usage: 11.8 MB


In [18]:
continous_columns = ['city_development_index', 'training_hours']

In [19]:
categorical_columns = ['city', 'gender', 'relevent_experience', 'enrolled_university', 'education_level', 'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job']

In [20]:
df[categorical_columns] = df[categorical_columns].astype('category')

In [21]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19158 entries, 1 to 33380
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   city                    19158 non-null  category
 1   city_development_index  19158 non-null  float64 
 2   gender                  14650 non-null  category
 3   relevent_experience     19158 non-null  category
 4   enrolled_university     18772 non-null  category
 5   education_level         18698 non-null  category
 6   major_discipline        16345 non-null  category
 7   experience              19093 non-null  category
 8   company_size            13220 non-null  category
 9   company_type            13018 non-null  category
 10  last_new_job            18735 non-null  category
 11  training_hours          19158 non-null  int64   
 12  target                  19158 non-null  float64 
dtypes: category(10), float64(2), int64(1)
memory usage: 803.1 KB


In [39]:
df.dtypes

city                      category
city_development_index     float64
gender                    category
relevent_experience       category
enrolled_university       category
education_level           category
major_discipline          category
experience                category
company_size              category
company_type              category
last_new_job              category
training_hours               int64
target                     float64
dtype: object

In [40]:
df.head()

,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
enrollee_id,,,,,,,,,,,,,
1,city_103,0.920,Male,No relevent experience,no_enrollment,High School,NaN,2,NaN,Pvt Ltd,never,150,0.0
2,city_103,0.920,Male,No relevent experience,no_enrollment,Phd,STEM,15,10000+,NGO,3,128,1.0
4,city_103,0.920,Male,No relevent experience,no_enrollment,Masters,STEM,9,50-99,Public Sector,>4,13,0.0
5,city_67,0.855,Male,Has relevent experience,no_enrollment,Masters,STEM,10,<10,Early Stage Startup,1,12,0.0
7,city_136,0.897,Male,Has relevent experience,Full time course,Masters,STEM,13,NaN,NaN,1,18,0.0


### Verify what kind of unique values exist in each column:

In [18]:
for col in categorical_columns:
    unique_items = df[col].unique().tolist()
    print(f'col name: {col}, unique_items: {unique_items}')

col name: city, unique_items: ['city_103', 'city_67', 'city_136', 'city_16', 'city_114', 'city_21', 'city_23', 'city_73', 'city_104', 'city_10', 'city_100', 'city_46', 'city_105', 'city_97', 'city_74', 'city_61', 'city_57', 'city_160', 'city_65', 'city_45', 'city_72', 'city_41', 'city_44', 'city_40', 'city_131', 'city_98', 'city_115', 'city_118', 'city_71', 'city_20', 'city_159', 'city_173', 'city_64', 'city_28', 'city_102', 'city_84', 'city_101', 'city_11', 'city_27', 'city_36', 'city_50', 'city_13', 'city_144', 'city_91', 'city_145', 'city_165', 'city_134', 'city_152', 'city_24', 'city_162', 'city_39', 'city_37', 'city_1', 'city_106', 'city_83', 'city_150', 'city_99', 'city_80', 'city_142', 'city_33', 'city_138', 'city_75', 'city_53', 'city_128', 'city_9', 'city_30', 'city_12', 'city_141', 'city_116', 'city_157', 'city_175', 'city_25', 'city_149', 'city_123', 'city_19', 'city_76', 'city_89', 'city_143', 'city_70', 'city_158', 'city_90', 'city_126', 'city_42', 'city_14', 'city_77', 'c

In [41]:
for col in categorical_columns:
    unique_items = df[col].unique().tolist()
    print(f'col name: {col}, len of unique items: {len(unique_items)}')

col name: city, len of unique items: 123
col name: gender, len of unique items: 4
col name: relevent_experience, len of unique items: 2
col name: enrolled_university, len of unique items: 4
col name: education_level, len of unique items: 6
col name: major_discipline, len of unique items: 7
col name: experience, len of unique items: 23
col name: company_size, len of unique items: 9
col name: company_type, len of unique items: 7
col name: last_new_job, len of unique items: 7


### See how many nans exist in each column:

In [42]:
from pandas import isnull
df_is_na_value = df.applymap(lambda x: isnull(x))

In [43]:
df_is_na_value = df.apply(lambda x: x.isna())

In [44]:
df_is_na_value.sum(axis=0)

city                         0
city_development_index       0
gender                    4508
relevent_experience          0
enrolled_university        386
education_level            460
major_discipline          2813
experience                  65
company_size              5938
company_type              6140
last_new_job               423
training_hours               0
target                       0
dtype: int64

In [45]:
df_is_na_value.sum(axis=1)

enrollee_id
1        2
2        0
4        0
5        0
7        2
        ..
33375    0
33376    0
33377    0
33379    0
33380    0
Length: 19158, dtype: int64

# Question: What do we do with those nan values?

### A quick remark on speed issues in pandas
Let's assume we want to perform a very simple operation - we need to raise the values in the 'city_development_index' column to the second power. Let's try three approaches:
- loops
- apply() method
- builtin vectorized pandas method

##### Loop

In [46]:
%%timeit

for _, record in df.iterrows():
    record['city_development_index'] ** 2

453 ms ± 47.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


##### apply()

In [47]:
%%timeit

df['city_development_index'].apply(lambda x: x**2)

2.64 ms ± 226 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


##### vectorized builtin method

In [48]:
%%timeit

df['city_development_index'] ** 2

50.9 µs ± 4.96 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### One Hot Encoding - what it is?

In [22]:
from sklearn.preprocessing import OneHotEncoder

In [23]:
encoder = OneHotEncoder()

In [24]:
encoder.fit(df[categorical_columns])

OneHotEncoder()

In [25]:
encoded = encoder.transform(df[categorical_columns])

In [26]:
encoded.shape

(19158, 192)

In [30]:
type(encoded)

scipy.sparse._csr.csr_matrix

In [34]:
encoded.toarray()

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [64]:
encoder.categories_

[array(['city_1', 'city_10', 'city_100', 'city_101', 'city_102',
        'city_103', 'city_104', 'city_105', 'city_106', 'city_107',
        'city_109', 'city_11', 'city_111', 'city_114', 'city_115',
        'city_116', 'city_117', 'city_118', 'city_12', 'city_120',
        'city_121', 'city_123', 'city_126', 'city_127', 'city_128',
        'city_129', 'city_13', 'city_131', 'city_133', 'city_134',
        'city_136', 'city_138', 'city_139', 'city_14', 'city_140',
        'city_141', 'city_142', 'city_143', 'city_144', 'city_145',
        'city_146', 'city_149', 'city_150', 'city_152', 'city_155',
        'city_157', 'city_158', 'city_159', 'city_16', 'city_160',
        'city_162', 'city_165', 'city_166', 'city_167', 'city_171',
        'city_173', 'city_175', 'city_176', 'city_179', 'city_18',
        'city_180', 'city_19', 'city_2', 'city_20', 'city_21', 'city_23',
        'city_24', 'city_25', 'city_26', 'city_27', 'city_28', 'city_30',
        'city_31', 'city_33', 'city_36', 'cit

In [65]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [66]:
encoded.shape

(19158, 192)

In [67]:
from sklearn.model_selection import train_test_split

In [68]:
encoded.toarray()

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [69]:
continous_values = df[continous_columns].values

In [70]:
continous_values

array([[  0.92 , 150.   ],
       [  0.92 , 128.   ],
       [  0.92 ,  13.   ],
       ...,
       [  0.802,  27.   ],
       [  0.92 ,  81.   ],
       [  0.923, 168.   ]])

In [71]:
continous_values.shape

(19158, 2)

In [72]:
import numpy as np

In [73]:
print(encoded.toarray().shape)
print(continous_values.shape)

(19158, 192)
(19158, 2)


In [74]:
x_values = np.concatenate(
    (encoded.toarray(), continous_values), axis=1
)

In [75]:
x_values.shape

(19158, 194)

In [76]:
y_values = df['target'].values

In [77]:
y_values

array([0., 1., 0., ..., 0., 0., 0.])

In [78]:
y_values.shape

(19158,)

In [79]:
x_train, x_val, y_train, y_val = train_test_split(x_values, y_values)

In [80]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)


(14368, 194)
(14368,)
(4790, 194)
(4790,)


In [81]:
model = LogisticRegression()

In [82]:
model.fit(x_train, y_train)

/Users/patrykseweryn/Library/Python/3.8/lib/python/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [83]:
y_pred = model.predict(x_val)

In [84]:
y_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [85]:
from sklearn.metrics import accuracy_score, f1_score, recall_score

In [86]:
y_pred_train = model.predict(x_train)

In [87]:
train_acc = accuracy_score(y_train, y_pred_train)
val_acc = accuracy_score(y_val, y_pred)

In [88]:
print(f'training accuracy: {train_acc}')

training accuracy: 0.7820155902004454


In [89]:
print(f'validation accuracy: {val_acc}')

validation accuracy: 0.7839248434237995


In [90]:
train_f1 = f1_score(y_train, y_pred_train)
val_f1 = f1_score(y_val, y_pred)

In [91]:
train_f1

0.45148861646234684

In [117]:
val_f1

0.5485611510791367

In [43]:
pipeline = Pipeline([
    ('encoder', OneHotEncoder()),
    ('classifier', LinearRegression())]
)

In [92]:
from sklearn.metrics import classification_report

In [98]:
classification_report(y_val, y_pred, output_dict=True, target_names=['not leaving', 'leaving'])

{'not leaving': {'precision': 0.813079551000488,
  'recall': 0.9252985281866148,
  'f1-score': 0.8655669567476296,
  'support': 3601},
 'leaving': {'precision': 0.611271676300578,
  'recall': 0.35576114381833474,
  'f1-score': 0.44976076555023925,
  'support': 1189},
 'accuracy': 0.7839248434237995,
 'macro avg': {'precision': 0.7121756136505331,
  'recall': 0.6405298360024748,
  'f1-score': 0.6576638611489345,
  'support': 4790},
 'weighted avg': {'precision': 0.762985696508172,
  'recall': 0.7839248434237995,
  'f1-score': 0.7623532696215968,
  'support': 4790}}

#### What can you tell about those results? Are we satisfied with this approach? Can you please try other models?

* RandomForestClassifier
* XGBClassifier
* DecisionTree

### How can we manipulate with the data? What can be done? Please suggest mainly what can be done with:

* high cardinality of the data
* NaN values

### Explain the results - Eli5

In [119]:
import eli5